In [1]:
from azureml.core import Workspace
 
print("Accessing the workspace...")
ws = Workspace.from_config(path="./config")

print("Accessing the dataset...")
input_ds = ws.datasets.get('default')


# --------------------------------------------------------------------
# Create the compute Cluster 
# --------------------------------------------------------------------
cluster_name = "my-cluster-001"

from azureml.core.compute import AmlCompute

print("Accessing the compute cluster...")

# cluster が存在している場合はそれを使う
if cluster_name not in ws.compute_targets:
    print("Creating the compute cluster with name: ", cluster_name)
    compute_config = AmlCompute.provisioning_configuration(
                                     vm_size="STANDARD_D11_V2",
                                     max_nodes=2)

    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster = ws.compute_targets[cluster_name]
    print(cluster_name, ", compute cluster found. Using it...")



# --------------------------------------------------------------------
# Configure the AutoML run
# --------------------------------------------------------------------

from azureml.train.automl import AutoMLConfig

print("Creating the AutoML Configuration...")

# norm_macro_recallについては動画参照
automl_config = AutoMLConfig(task='classification',
                             compute_target=cluster,
                             training_data=input_ds,
                             validation_size=0.3,
                             label_column_name='Default Next Month',
                             primary_metric='norm_macro_recall',
                             iterations=10,
                             max_concurrent_iterations=2, # nodeのmax数まで指定可能
                             experiment_timeout_hours=0.25,
                             featurization='auto')

# --------------------------------------------------------------------
# Create and submit the experiment
# --------------------------------------------------------------------

from azureml.core.experiment import Experiment

new_exp = Experiment(ws, 'azureml-sdk-exp-001')

print("Submitting the experiment....")

new_run = new_exp.submit(automl_config)
new_run.wait_for_completion(show_output=True)

Accessing the workspace...


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Accessing the dataset...
Accessing the compute cluster...
my-cluster-001 , compute cluster found. Using it...
Creating the AutoML Configuration...
Submitting the experiment....
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
azureml-sdk-exp-001,AutoML_f04761f1-d26c-4036-a234-30f51f932e1b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
azureml-sdk-exp-001,AutoML_f04761f1-d26c-4036-a234-30f51f932e1b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of mi

{'runId': 'AutoML_f04761f1-d26c-4036-a234-30f51f932e1b',
 'target': 'my-cluster-001',
 'status': 'Completed',
 'startTimeUtc': '2022-10-04T14:49:36.930827Z',
 'endTimeUtc': '2022-10-04T14:54:01.546179Z',
 'services': {},
 'properties': {'num_iterations': '10',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0.3',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'my-cluster-001',
  'AMLSettingsJsonString': '{"path":null,"name":"azureml-sdk-exp-001","subscription_id":"3467f739-a57b-4612-9de8-72a6616c01b3","resource_group":"AzuremlSDKRG00","workspace_name":"Azureml-SDK-WS01","region":"japaneast","compute_target":"my-cluster-001","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code

In [2]:
# Retrieve the best model
# accuracyがbestのモデルを取得可能
new_run.get_best_child(metric='accuracy')

Experiment,Id,Type,Status,Details Page,Docs Page
azureml-sdk-exp-001,AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_6,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
# Get the metrics for all the runs
for run in new_run.get_children():
    print("")
    print("Run ID : ", run.id)
    print(run.get_metrics('accuracy'))
    print(run.get_metrics('norm_macro_recall'))


Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_8
{'accuracy': 0.7125}
{'norm_macro_recall': 0.24568661083644439}

Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_9
{'accuracy': 0.5895833333333333}
{'norm_macro_recall': 0.2163757441226919}

Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_3
{'accuracy': 0.6208333333333333}
{'norm_macro_recall': 0.1821208757945718}

Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_7
{'accuracy': 0.7666666666666667}
{'norm_macro_recall': 0.09888003228735753}

Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_2
{'accuracy': 0.775}
{'norm_macro_recall': 0.11633538492583995}

Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_6
{'accuracy': 0.7791666666666667}
{'norm_macro_recall': 0.01352033094541416}

Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_1
{'accuracy': 0.7541666666666667}
{'norm_macro_recall': 0.12339824437493685}

Run ID :  AutoML_f04761f1-d26c-4036-a234-30f51f932e1b_5
{'accuracy': 0.7166666666666667}
{'norm